In [47]:
class job:
    def __init__(self, name, machines_to_use, runtime_list):
        self.name = name
        self.machines_to_use = machines_to_use
        self.runtime_list = runtime_list
        self.step = -1
        
    def get_next_machine(self):
        self.step += 1
        if self.step < 4:
            next_machine = self.machines_to_use[self.step]
            return next_machine
        else:
            return None
        
    def get_runtime(self, machine_name):
        for i in range(len(self.machines_to_use)):
            if self.machines_to_use[i].name == machine_name:
                return self.runtime_list[i]
            
        print("error trying to find runtime for machine {}".format(machine_name))
        
        return 0
        
    

In [53]:
class Machine:
    def __init__(self, name, setup):
        self.name = name
        self.setup = setup
        self.running = False
        self.time_to_complete = 0
        self.current_job = None
        self.waitlist = []
        
    def add_starting_jobs(self,job_list):
        self.waitlist = job_list
        
    def add_job(self, job):
        self.waitlist.append(job)
        self.update(0)
    
    def run_machine(self, job):
        if (self.running and job) and job not in self.waitlist:
            self.waitlist.append(job)
        else:
            if self.current_job:
                if self.current_job.name == job.name:
                    # did not change job, no setup needed
                    self.time_to_complete = job.get_runtime(self.name)
                else:
                    self.time_to_complete = job.get_runtime(self.name) + self.setup
                    self.current_job = job
            else:
                self.time_to_complete = job.get_runtime(self.name)
                self.current_job = job
            
            print("machine {} just started job {}".format(self.name, self.current_job.name))
                
            self.running = True
        
    def update(self, t):
        if not self.running and not self.waitlist:
            return None
        elif not self.running: 
            self.run_machine(self.waitlist[0])
            self.waitlist = self.waitlist[1:]
            return None
        
        if self.time_to_complete <= t:
            print("machine {} just completed job {}".format(self.name, self.current_job.name))
            completed_job = self.current_job
            self.time_to_complete = 0
            self.current_job = None
            self.running = False
            
            if self.waitlist:
                #FIFO logic
                self.run_machine(self.waitlist[0])
                self.waitlist = self.waitlist[1:]
            
            return completed_job
        
        else:
            #print("machine {} working on job {}".format(self.name, self.current_job.name))
            self.time_to_complete -= t
            #print("time_to_completion: {}".format(self.time_to_complete))
            return None
        
        
    

In [54]:
c1, c2, c3 = Machine("Chucker1", 1.5), Machine("Chucker2", 1.5), Machine("Chucker3", 1.5)
ca, cb, cc, cd = Machine("ChuckerA", 1.5), Machine("ChuckerB", 1.5), Machine("ChuckerC", 1.5), Machine("ChuckerD", 1.5)
m1, m2 = Machine("Mill1", 2), Machine("Mill2", 2)
n = Machine("Natco Drill", 3)

In [58]:
# dedicated line for B 15
runtime = [6,8,4,1.5]
setup = [0,0,0,0]
backlog = 58
machines =[c3,cd,m2,n]
jobs_b15 = [job("B15_"+str(j), [c3,cd,m2,n], runtime) for j in range(backlog)]


In [59]:
max_time = 9999
passed_time = 0
t = 0.5
c3.add_starting_jobs(jobs_b15)
c3.update(0)
have_remaining_jobs = True
while passed_time < max_time and have_remaining_jobs:
    passed_time+=t

    for m in machines:
        m_completed = m.update(t)
        if m_completed:
            next_m = m_completed.get_next_machine()
            if next_m:
                next_m.add_job(m_completed)
    
    if c3.waitlist or cd.waitlist or m2.waitlist or n.waitlist:
        have_remaining_jobs = True
    else:
        have_remaining_jobs = False
    
print("total hours used: {}".format(passed_time))
    

machine Chucker3 just completed job B15_7
machine Chucker3 just started job B15_0
machine Mill2 just completed job B15_5
machine Natco Drill just started job B15_5
machine Natco Drill just completed job B15_5
machine ChuckerD just completed job B15_6
machine ChuckerD just started job B15_7
machine Mill2 just started job B15_6
machine Chucker3 just completed job B15_0
machine Chucker3 just started job B15_1
machine Mill2 just completed job B15_6
machine Natco Drill just started job B15_6
machine Natco Drill just completed job B15_6
machine Chucker3 just completed job B15_1
machine Chucker3 just started job B15_2
machine ChuckerD just completed job B15_7
machine Mill2 just started job B15_7
machine Mill2 just completed job B15_7
machine Natco Drill just started job B15_7
machine Chucker3 just completed job B15_2
machine Chucker3 just started job B15_3
machine Natco Drill just completed job B15_7
machine Chucker3 just completed job B15_3
machine Chucker3 just started job B15_4
machine Chu